In [1]:
import pandas as pd
import pickle
station_coord = pd.read_csv('Station_coord.csv')
station_coord

,Unnamed: 0,Station,Adresse,lat,long,x,y
0,0,Barking,"Alfred's Way, IG11 0BB",51.529820,0.088809,9886.172658,6.715553e+06
1,1,Dagenham,"70 Rainham Road North, Dagenham, RM10 7ES",51.559507,0.156780,17452.669767,6.720867e+06
2,2,Dowgate,"94-95 Upper Thames Street, EC4R 3UE",51.510003,-0.090206,-10041.685986,6.712008e+06
3,3,Shoreditch,"235 Old Street, EC1V 9EY",51.526625,-0.084321,-9386.570783,6.714982e+06
4,4,Stoke Newington,"64 Stoke Newington Church Street, N16 0AR",51.562056,-0.076530,-8519.280630,6.721324e+06
...,...,...,...,...,...,...,...
98,98,Wandsworth,"45 West Hill Wandsworth, SW18 1RL",51.456521,-0.201461,-22426.535935,6.702448e+06
99,99,Battersea,"11 Este Road, SW11 2TL",51.467349,-0.168972,-18809.876998,6.704382e+06
100,100,Tooting,"91 Trinity Road, SW17 7SQ",51.437991,-0.162768,-18119.250877,6.699138e+06
101,101,Paddington,"156 Harrow Rd, London, W2 6NL, UK",51.520219,-0.183039,-20375.808275,6.713835e+06


In [2]:
import numpy as np
def wgs84_to_web_mercator(df, lon="LON", lat="LAT"):
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k

    return df
station_coord = wgs84_to_web_mercator(station_coord, 'long', 'lat')
station_coord

,Unnamed: 0,Station,Adresse,lat,long,x,y
0,0,Barking,"Alfred's Way, IG11 0BB",51.529820,0.088809,9886.172658,6.715553e+06
1,1,Dagenham,"70 Rainham Road North, Dagenham, RM10 7ES",51.559507,0.156780,17452.669767,6.720867e+06
2,2,Dowgate,"94-95 Upper Thames Street, EC4R 3UE",51.510003,-0.090206,-10041.685986,6.712008e+06
3,3,Shoreditch,"235 Old Street, EC1V 9EY",51.526625,-0.084321,-9386.570783,6.714982e+06
4,4,Stoke Newington,"64 Stoke Newington Church Street, N16 0AR",51.562056,-0.076530,-8519.280630,6.721324e+06
...,...,...,...,...,...,...,...
98,98,Wandsworth,"45 West Hill Wandsworth, SW18 1RL",51.456521,-0.201461,-22426.535935,6.702448e+06
99,99,Battersea,"11 Este Road, SW11 2TL",51.467349,-0.168972,-18809.876998,6.704382e+06
100,100,Tooting,"91 Trinity Road, SW17 7SQ",51.437991,-0.162768,-18119.250877,6.699138e+06
101,101,Paddington,"156 Harrow Rd, London, W2 6NL, UK",51.520219,-0.183039,-20375.808275,6.713835e+06


In [3]:
from bokeh.plotting import figure
from bokeh.io import  push_notebook,output_notebook, show
from  bokeh.models import ColumnDataSource, LabelSet
from bokeh.tile_providers import get_provider
tuile = get_provider('STAMEN_TERRAIN_RETINA')
station_coord = wgs84_to_web_mercator(station_coord, 'long', 'lat')
source = ColumnDataSource(station_coord)
# On récupère les valeurs mini/maxi des coordonnées,
# on rajoute 10km pour avoir une carte centrée
xutm_min, xutm_max, yutm_min, yutm_max = station_coord.x.min()-10000, station_coord.x.max()+10000, \
                                         station_coord.y.min()-10000, station_coord.y.max()+10000
print(xutm_min, xutm_max, yutm_min, yutm_max)
x_range, y_range, x_axis_type, y_axis_type = (xutm_min, xutm_max), (yutm_min, yutm_max), 'mercator', 'mercator'
p = figure(x_range=x_range, y_range=y_range, x_axis_type=x_axis_type, y_axis_type=y_axis_type,
          plot_width=1200, plot_height=1200)
p.add_tile(tuile)

from bokeh.models.tools import HoverTool
p.circle(x='x',y='y', size=15, fill_color='red', source=source)
p.xaxis.major_label_text_font_size = '25px'
p.xaxis.axis_label = "Lon"
p.xaxis.axis_label_text_font_size = '25px'

p.yaxis.major_label_text_font_size = '25px'
p.yaxis.axis_label = "Lat"
p.yaxis.axis_label_text_font_size = '25px'

#lab = LabelSet(x='x',y='y',text='Station',x_offset=0,y_offset=0, source=source, text_align="center")
#p.add_layout(lab)
show(p)

-61054.45806251906 35926.30940575342 6666977.681650863 6748930.736050663
